In [ ]:
from pyhive import hive
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import pandasql as ps
from pandasql import sqldf
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(font_scale=3)
sns.set(color_codes=True)
%matplotlib inline

from sklearn.manifold import TSNE
from sklearn import preprocessing
from sklearn import model_selection
import shap


In [ ]:
def replace_space_in_col_names(df):
    new_cols = [x.replace(" ","_") for x in df.columns]
    df.columns = new_cols

In [ ]:
trans_df = pd.read_csv("../quva_trans.csv")
trans_df["Customer_ID"] = trans_df["Customer_ID"].astype(str)
trans_df["Order_Date"] = pd.to_datetime(trans_df["Order_Date"])
cust_first_trans_date_df =trans_df.groupby(["Customer_ID"]).min()["Order_Date"].reset_index()
cust_first_trans_date_df.columns = ["Customer_ID","First_Order_Date"]
trans_df = pd.merge(trans_df,cust_first_trans_date_df)
trans_df["First_Order_Date"] = pd.to_datetime(trans_df["First_Order_Date"])
trans_df["year_month"] = trans_df["Order_Date"].apply(lambda x : str(x.year) + str(x.month).zfill(2)) 

In [ ]:
sf_case_df = pd.read_excel("../../data/QuVa-GAIL DataRequest_6_7_8.xlsx","SF_Case")
replace_space_in_col_names(sf_case_df)

sf_case_df["Customer_id"] = sf_case_df["Customer_id"].astype(str)
sf_case_df.CreatedDate = pd.to_datetime(sf_case_df.CreatedDate).dt.to_period('D')
sf_case_df.ClosedDate = pd.to_datetime(sf_case_df.ClosedDate).dt.to_period('D')
sf_case_df["Closed_Year_Month"] = sf_case_df["ClosedDate"].apply(lambda x : str(x.year) + str(x.month).zfill(2)) 

#filter out the customers with zero transactions
sf_case_df = sf_case_df[sf_case_df.Customer_id.isin(trans_df.Customer_ID)]
sf_case_df_copy = sf_case_df.copy()
#sf_case_df = sf_case_df[["CaseId","CreatedDate","ClosedDate","Closed_Year_Month","Customer_id","Type","Priority","Reason"]]
sf_case_df = sf_case_df[["CaseId","CreatedDate","ClosedDate","Closed_Year_Month","Customer_id","Type"]]
sf_case_df = sf_case_df.sort_values(by=["Customer_id","Closed_Year_Month"]).reset_index().drop('index',axis=1)

In [ ]:
sns.distplot(sf_case_df.groupby(["Customer_id"]).count()["CaseId"],bins=20)

In [ ]:
sf_case_features_df = sf_case_df.copy()
sf_case_features_df = pd.get_dummies(sf_case_features_df, columns=['Type'])
sf_case_features_df_case_counts = sf_case_features_df.groupby(["Customer_id","Closed_Year_Month"]).count()["CaseId"].reset_index()
sf_case_features_df_type_counts = sf_case_features_df.groupby(["Customer_id","Closed_Year_Month"]).sum().reset_index()
sf_case_features_df_counts = pd.merge(sf_case_features_df_case_counts,sf_case_features_df_type_counts)
sf_case_features_df_counts.head()
sf_case_features_df_counts_cum = sf_case_features_df_counts[["Customer_id","Closed_Year_Month"]]
sf_case_features_df_counts_cum = pd.concat([sf_case_features_df_counts_cum,sf_case_features_df_counts.groupby(["Customer_id"]).cumsum()] ,axis=1)
sf_case_features_df_counts_cum = sf_case_features_df_counts_cum[sf_case_features_df_counts_cum.Closed_Year_Month!='nannan']



In [ ]:
#group Customers trans by year/month
trans_ym_df = trans_df[["Customer_ID","year_month","Daily_Revenue"]].groupby(["Customer_ID","year_month"]).sum().reset_index().sort_values(by=["Customer_ID","year_month"])
trans_ym_df["Rev_Next_3months"] = trans_ym_df.groupby(["Customer_ID"])["Daily_Revenue"].rolling(3).sum().reset_index(0,drop=True).shift(-3)
#sf_case_trans_joined = pd.merge(sf_case_features_df_counts,trans_df,on=

trans_ym_df.head()

In [ ]:
trans_cases_joined_df.columns

In [ ]:
print(len(trans_cases_joined_df.Customer_id.unique()))
significant_count_pos = 0
significant_count_neg = 0
for g in trans_cases_joined_df.groupby('Customer_ID')[['Rev_Next_3months',"CaseId"]]:
    current_corr_pvalue = stats.pearsonr(g[1]["Rev_Next_3months"], g[1]["CaseId"])
    if current_corr_pvalue[1] <= 0.05 and current_corr_pvalue[0] > 0:
        significant_count_pos = significant_count_pos + 1
    elif current_corr_pvalue[1] <= 0.05 and current_corr_pvalue[0] < 0:
        significant_count_neg = significant_count_neg + 1
        
print(significant_count_neg)
print(significant_count_pos)


In [ ]:
import scipy.stats  as stats
trans_cases_joined_df = pd.merge(trans_ym_df,sf_case_features_df_counts_cum,left_on=["Customer_ID","year_month"],right_on=["Customer_id","Closed_Year_Month"])
trans_cases_joined_df = trans_cases_joined_df[trans_cases_joined_df.year_month.astype(int) <= 201804]
#trans_cases_joined_df["Rev_Next_3months"].corr(trans_cases_joined_df["CaseId"])

cust_cases_corr_df = pd.DataFrame()
c = 'CaseId'
cust_cases_corr_df = trans_cases_joined_df.groupby('Customer_ID')[['Rev_Next_3months',c]].corr().round(3).dropna().reset_index()[['Customer_ID',c]]
cust_cases_corr_df = cust_cases_corr_df[cust_cases_corr_df[c]!=1]
for c in trans_cases_joined_df.columns:
    if 'Type_' in c:
        temp_df = trans_cases_joined_df.groupby('Customer_ID')[['Rev_Next_3months',c]].corr().round(3).dropna().reset_index()[['Customer_ID',c]]
        temp_df = temp_df[temp_df[c] !=1]
        if len(cust_cases_corr_df) == 0:
            cust_cases_corr_df = temp_df
        else:
            cust_cases_corr_df = pd.merge(cust_cases_corr_df,temp_df,how='left')

cust_cases_corr_df = cust_cases_corr_df.fillna(0)
cust_cases_corr_df.head()



In [ ]:
cust_joined_with_neg_flag_df = trans_cases_joined_df.copy()
cust_joined_with_neg_flag_df["Is_Declining"] = cust_joined_with_neg_flag_df.Customer_ID.apply(lambda x : 1 if x in list(cust_cases_corr_df[cust_cases_corr_df.CaseId < -0.01 ].Customer_ID) else 0)
cust_joined_with_neg_flag_df["trans_id"] = cust_joined_with_neg_flag_df["Customer_ID"] + "_" + cust_joined_with_neg_flag_df["year_month"]
cust_joined_with_neg_flag_df.groupby(["Is_Declining"]).agg({'Customer_ID':'nunique',
                                                           'Daily_Revenue':'sum',
                                                           'CaseId' : 'max',
                                                           'trans_id' :'nunique'})


In [ ]:
cust_joined_with_neg_flag_df.sort_values(by=["Customer_ID","year_month"])[["Customer_ID","year_month","Daily_Revenue","Rev_Next_3months","CaseId"]].tail(20)

In [ ]:
#sns.distplot(cust_cases_corr_df.Type_Portal)
corr_melt = pd.melt(cust_cases_corr_df, "Customer_ID")
g = sns.FacetGrid(corr_melt, col="variable", col_wrap=7, sharex=False,gridspec_kws={"hspace":0.9})
g.map(plt.hist, "value", alpha=.4)

In [ ]:
ax = sns.distplot(cust_cases_corr_df.CaseId,bins=10,kde=False,kde_kws={'clip': (0.0, 1.0)})
ax.set_ylabel("Number of Customers")
ax.set_xlabel("Correlation with the total number of cases")

In [ ]:
sample_custs = trans_cases_joined_df.groupby('Customer_ID').filter(lambda x : len(x) > 5 )['Customer_ID']
cust_cases_corr_df_2 = cust_cases_corr_df[cust_cases_corr_df.Customer_ID.isin(sample_custs)]
high_case_id_customers = cust_cases_corr_df_2.sort_values(by='CaseId',ascending=False)['Customer_ID'].head(5)
high_case_id_customers_joined = trans_cases_joined_df[trans_cases_joined_df.Customer_ID.isin(high_case_id_customers)]
high_case_id_customers_joined
print(high_case_id_customers_joined.Customer_ID.drop_duplicates().head(2))

for c in high_case_id_customers_joined.Customer_ID.drop_duplicates():
    ax = sns.lineplot(label="Customer_ID:" + c, y="Rev_Next_3months", x="CaseId", data=high_case_id_customers_joined[high_case_id_customers_joined.Customer_ID == c])
    ax.set_ylabel('Total Revenue in the next 3 months')
    ax.set_xlabel('Number of cases over time')
    plt.legend(loc='upper right')

#sns.lineplot(y="Rev_Next_3months", x="CaseId", data=high_case_id_customers_joined[high_case_id_customers_joined.Customer_ID == "10172"])

In [ ]:
high_case_id_customers_joined[high_case_id_customers_joined.Customer_ID=="10163"]
high_case_id_customers_joined

In [ ]:
sf_account_pid_mapping_df = pd.read_csv("../QuVa_Featurizer/sf_account_pid_mapping_df.csv")[["Account_id","Provider_ID","DUET__extrecordid__c"]]
sf_account_pid_mapping_df.rename(columns={'DUET__extrecordid__c' : 'Customer_ID'},inplace=True)
sf_account_pid_mapping_df.head()
cust_cases_corr_df_joined = pd.merge(cust_cases_corr_df,sf_account_pid_mapping_df)
cust_cases_corr_df_joined["Is_Declining"] = cust_cases_corr_df_joined["CaseId"].apply(lambda x : 1 if x < -0.01 else 0)
cust_cases_corr_df_joined = cust_cases_corr_df_joined[["Customer_ID","Account_id","Provider_ID","CaseId","Is_Declining"]]

#get QuVa scaled features
quva_features_scaled = pd.read_csv("../QuVa_Featurizer/x_scaled_df_to_save.csv")
quva_features_scaled_joined = pd.merge(cust_cases_corr_df_joined,quva_features_scaled)

In [ ]:
len(quva_features_scaled_joined)

In [ ]:
X = quva_features_scaled_joined.drop(["Account_id","Provider_ID","Customer_ID","Is_Declining","CaseId"],axis=1)
y = quva_features_scaled_joined["Is_Declining"]
tsne = TSNE(n_components=2, verbose=1, perplexity=30, n_iter=800)
tsne_results = tsne.fit_transform(X)
tsne_df = pd.DataFrame(tsne_results)
tsne_df.columns = ["tsne1","tsne2"]
tsne_df["Is_Negative"] = y

sns.set(style="ticks")
g = sns.catplot(x="tsne1", y="tsne2", hue="Is_Negative", data=tsne_df)


In [ ]:
import lightgbm as lgb
def run_lgb(train_X, train_y,num_iterations=1000):
    params = {
        #"objective" : "quantile",
        "objective" : "binary",
        "metric" : "binary_logloss",
        'unbalance': True,
        "num_leaves" : 30,
        "learning_rate" : 0.01,
        "bagging_fraction" : 0.9,
        "feature_fraction" : 0.8,
        #"bagging_frequency" : 5,
        "bagging_seed" : 13
        #'metric': 'rmsle'
    }
    lgtrain = lgb.Dataset(train_X, label=train_y)
    model = lgb.train(params, lgtrain, num_iterations, verbose_eval=50)
    return model

In [ ]:
from sklearn.metrics import roc_auc_score,average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score,average_precision_score,f1_score

kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=113)
roc_auc_scores = []
avg_precision_scores=[]
for train_index, test_index in kf.split(x_scaled_df):
    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    model = run_lgb(X_train,y_train)
    y_predict = model.predict(X_test)
    #print(classification_report(y_test, y_predict.round(),target_names=["Prospect","Customer"]))
    roc_auc_scores.append(roc_auc_score(y_test,y_predict))
    avg_precision_scores.append(average_precision_score(y_test,y_predict))


print(np.average(roc_auc_scores))
print(np.average(avg_precision_scores))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=117)
model = run_lgb(X_train,y_train)
y_predict = model.predict(X_test)

print(classification_report(y_test, y_predict.round(),target_names=["Non_Declining","Is_Declining"]))
print(roc_auc_score(y_test,y_predict))
print(average_precision_score(y_test,y_predict))

In [ ]:
lgb.plot_importance(model, max_num_features=10)
fi=pd.DataFrame(model.feature_importance()).T
fi.columns = X.columns
fi = fi.T.reset_index()
fi.columns = ["Feature","Importance_Score"]
#fi.sort_values(by="Importance_Score").to_csv("light_gbm_fi.csv",index=False)


In [ ]:
shap.initjs()
# explain the model's predictions using SHAP values
# (same syntax works for LightGBM, CatBoost, and scikit-learn models)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
# visualize the first prediction's explanation
#shap.force_plot(explainer.expected_value, shap_values[0,:], x_scaled_df.iloc[0,:])
#shap.summary_plot(shap_values, x_scaled_df,max_display=len(x_scaled_df.columns),show=False,title="QuVa Prediction Features")
shap.summary_plot(shap_values, X,max_display=30,show=False,title="QuVa Prediction Features")

In [ ]:
len(X)

In [ ]:
def get_SHAP_features_importance(shap_values,all_features_names):
    global_shap_values = np.abs(shap_values).mean(0)
    shap_df = pd.DataFrame([global_shap_values],columns=all_features_names).T.reset_index()
    #shap_df.columns = list(x_scaled_df.columns)
    shap_df.columns = ["feature","mean_shap_value"]
    shap_df = shap_df.sort_values(by="mean_shap_value",ascending=False)
    return shap_df 

In [ ]:
get_SHAP_features_importance(shap_values,X.columns).to_csv("declining_model_fi.csv",index=False)

In [ ]:
sf_case_comments_df = pd.read_excel("../../data/QuVa-GAIL DataRequest_6_7_8.xlsx","SF_CaseComment")
sf_case_comments_df.CreatedDate = pd.to_datetime(sf_case_comments_df.CreatedDate).dt.to_period('D')
sf_case_df_copy = sf_case_df_copy.sort_values(by=['Customer_id','CaseId','CreatedDate'])
sf_case_df_copy2 = pd.merge(sf_case_df_copy,sf_case_comments_df,right_on='Caseid',left_on='CaseId',how='left')
high_case_id_customers = cust_cases_corr_df_2.sort_values(by='CaseId',ascending=True)['Customer_ID'].head(5)

sf_case_df_copy_neg = sf_case_df_copy2[sf_case_df_copy2.Customer_id.isin(high_case_id_customers)]
sf_case_df_copy_neg = sf_case_df_copy_neg[["CaseId","CreatedDate_x","Customer_id","Description","ClosedDate","Type","Resolution_Description__c","Resolution_Description","Reason","CommentBody"]]
sf_case_df_copy_neg["Is_Declining"] = 'True'

high_case_id_customers = cust_cases_corr_df_2.sort_values(by='CaseId',ascending=False)['Customer_ID'].head(5)

sf_case_df_copy_pos = sf_case_df_copy2[sf_case_df_copy2.Customer_id.isin(high_case_id_customers)]
sf_case_df_copy_pos = sf_case_df_copy_pos[["CaseId","CreatedDate_x","Customer_id","Customer_Name","Description","ClosedDate","Type","Resolution_Description__c","Resolution_Description","Reason","CommentBody"]]
sf_case_df_copy_pos["Is_Declining"] = 'False'



In [ ]:
sf_case_df_momentum_df = pd.concat([sf_case_df_copy_neg,sf_case_df_copy_pos],ignore_index=True,sort=False)
print(len(sf_case_df_copy_neg))
print(len(sf_case_df_copy_pos))
print(len(sf_case_df_momentum_df))
sf_case_df_momentum_df.rename(columns={'CreatedDate_x' : 'CreatedDate'},inplace=True)
sf_case_df_momentum_df.to_csv("sf_case_customers_examples.csv",index=False)

In [ ]:
trans_df["Customer_Age_In_Days"] = (trans_df["Order_Date"] - trans_df["First_Order_Date"]).apply(lambda x : x.days)
trans_df["Order_Year"] = trans_df.Order_Date.apply(lambda x : x.year)
trans_df["Order_Month"] = trans_df.Order_Date.apply(lambda x : x.month)
trans_df["Order_Day"] = trans_df.Order_Date.apply(lambda x : x.day)
trans_df = trans_df.sort_values(by=["Customer_ID","Order_Date"])

In [ ]:
trans_df = trans_df.drop(["New_TherClass_Flag"],axis=1,errors='ignore')
trans_df.to_csv("QuVa_Transactions_with_Customer_Age.csv",index=False)

In [ ]:
trans_df["Order_Month"] =  trans_df["Order_Month"].astype(str).apply(lambda x : x.zfill(2))
trans_df["year_month"] = trans_df["Order_Year"].astype(str) + trans_df["Order_Month"].astype(str)
trans_df["year_month"] = trans_df["year_month"].astype(int)
#sns.distplot(trans_df.groupby(["Customer_ID"]).nunique()["year_month"],hist_kws={'cumulative': True},kde_kws={'cumulative': True})
trans_df_12mon_cust = trans_df.groupby(["Customer_ID"]).nunique()["year_month"].reset_index()
trans_df_12mon_cust = trans_df_12mon_cust[trans_df_12mon_cust.year_month >= 12]
len(trans_df_12mon_cust)
trans_df_12mon_df = trans_df[trans_df.Customer_ID.isin( trans_df_12mon_cust["Customer_ID"])]
trans_df_12mon_df = trans_df_12mon_df.groupby(["Customer_ID","year_month"]).sum()["Daily_Revenue"].reset_index()
trans_df_12mon_df["cust_rank"] = trans_df_12mon_df.groupby(["Customer_ID"])["year_month"].rank()
trans_df_12mon_df = trans_df_12mon_df[trans_df_12mon_df["cust_rank"] <= 12]
#trans_df_12mon_df = trans_df_12mon_df.groupby(["Customer_ID","year_month"]).sum()["Daily_Revenue"].reset_index()




In [770]:
#timeseries clustering attempt
from dtw import dtw
import random
from numpy import array
from numpy.linalg import norm
from math import sqrt
import time
import datetime
x = array([0, 0, 1, 1, 2, 4, 2, 1, 2, 0]).reshape(-1, 1)
y = array([1, 1, 1, 2, 2, 2, 2, 3, 2, 0]).reshape(-1, 1)

dist, cost, acc, path = dtw(x, y, dist=lambda x, y: norm(x - y, ord=1))
print(dist)


def DTWDistance(s1, s2,w):
    DTW={}

    w = max(w, abs(len(s1)-len(s2)))

    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return sqrt(DTW[len(s1)-1, len(s2)-1])



def LB_Keogh(s1,s2,r):
    LB_sum=0
    for ind,i in enumerate(s1):

        lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])

        if i>upper_bound:
            LB_sum=LB_sum+(i-upper_bound)**2
        elif i<lower_bound:
            LB_sum=LB_sum+(i-lower_bound)**2

    return sqrt(LB_sum)





def k_means_clust(data,num_clust,num_iter,w=6):
    data_indexed = []
    for advisor_id in data:
        data_indexed.append((advisor_id,data[advisor_id]))
    
    sample_centroids=random.sample(list(data),num_clust)
    centroids = [] 
    for customer_id in sample_centroids:
        centroids.append((customer_id,data[customer_id]))
    counter=0
    for n in range(num_iter):
        counter = counter + 1
        current_time = time.time() #current timestamp
        ts_friendly = datetime.datetime.fromtimestamp(current_time).strftime('%Y_%m_%d %H:%M')
        assignments={}
        #assign data points to clusters
        for ind,i in enumerate(data_indexed):
        #for ind,i in data.iteritems():
            #i = data[i]
            data_points1 = array(i[1]).reshape(-1, 1)
            min_dist=float('inf')
            closest_clust=None
            for c_ind,j in enumerate(centroids):
                data_points2 = array(j[1]).reshape(-1, 1)
                #if LB_Keogh(data_points1,data_points2,5)<min_dist:
                cur_dist=DTWDistance(data_points1,data_points2,w)
                if cur_dist<min_dist:
                    min_dist=cur_dist
                    closest_clust=c_ind
            if closest_clust in assignments:
                assignments[closest_clust].append(i[0])
            else:
                assignments[closest_clust]=[i[0]]

        #recalculate centroids of clusters
        for key1 in assignments:
            clust_sum=0
            for k in assignments[key1]:
                #print(data[k])
                clust_sum=clust_sum+np.sum(data[k])
            
            #cluster_sum2 = [m/(len(assignments[key1])) for m in clust_sum]
            #centroids[key1]=[centroids[key1][0],cluster_sum2]
            centroids[key1]=[centroids[key1][0],clust_sum]

    return centroids , assignments





0.2


In [ ]:
trans_ym_df.head()

In [726]:
trans_ym_df_dense_groups = trans_ym_df.groupby('Customer_ID')
trans_ym_df_dense = trans_ym_df_dense_groups.filter(lambda x: len(x) >= 12).dropna()
trans_ym_df_dense["Rev_Next_3months_scaled"] = trans_ym_df_dense_groups["Rev_Next_3months"].transform(lambda x : (x-x.min())/(x.max()-x.min()))
trans_ym_df_dense = trans_ym_df_dense.drop(['Daily_Revenue','Rev_Next_3months'],axis=1)

trans_ym_df_dense.groupby('Customer_ID').count()["year_month"].max()
dummy_time_index_df = pd.DataFrame(np.arange(1,trans_ym_df_dense.groupby('Customer_ID').count()["year_month"].max() + 1))
dummy_time_index_df.columns=["time_index"]
dummy_time_index_df["time_index"] = dummy_time_index_df["time_index"].astype(int)


In [727]:
full_cust_time_index = pd.MultiIndex.from_product([list(dummy_time_index_df.time_index.unique()), list(trans_ym_df_dense.Customer_ID.unique())], names = ["time_index", "Customer_ID"])
dummy_time_index_df = pd.DataFrame(index=full_cust_time_index).reset_index()
dummy_time_index_df.Customer_ID.unique()

219

In [728]:
trans_ym_df_dense["time_index"] = trans_ym_df_dense.groupby('Customer_ID')['year_month'].cumcount() + 1
trans_ym_df_dense
trans_ym_df_dense["time_index"] = trans_ym_df_dense["time_index"].astype(int)
trans_ym_df_sparse = pd.merge(trans_ym_df_dense,dummy_time_index_df,how='outer')
trans_ym_df_sparse = trans_ym_df_sparse.sort_values(by=['Customer_ID','time_index']).reset_index()
trans_ym_df_sparse = trans_ym_df_sparse.drop(["index"],axis=1)

#fill missing values with moving average
#trans_ym_df_sparse.groupby('Customer_ID')["Rev_Next_3months_scaled"].fillna()
trans_ym_df_sparse["Rev_Next_3months_scaled"] = trans_ym_df_sparse.groupby('Customer_ID')["Rev_Next_3months_scaled"].transform(lambda x : x.fillna(x.mean()))
trans_ym_df_sparse = trans_ym_df_sparse[["Customer_ID","Rev_Next_3months_scaled"]]
trans_ym_df_sparse = trans_ym_df_sparse.dropna()
#trans_ym_df_sparse.set_index('Customer_ID')["Rev_Next_3months_scaled"].to_dict()["10002"]


In [766]:
def update_hash_map_list_values (hash_map , key , value):
    value_list = hash_map.get(key) #ts of a given customer
    if value_list == None:
        value_list = []
    value_list.append(value) 
    hash_map[key] = value_list

cust_ts = {}
for index, row in trans_ym_df_sparse.iterrows():
    update_hash_map_list_values(cust_ts,row["Customer_ID"],row["Rev_Next_3months_scaled"])

    



In [771]:
centroids , assignments =  k_means_clust(cust_ts,3,20,3)

In [747]:
random.sample(list(cust_ts),3)

['10204', '10132', '10002']

In [775]:
centroids

[['9451', 2091.9727903582684],
 ['10156', 2091.9727903582684],
 ['10284', 2091.9727903582684]]

dict_keys([0])